In [1]:
import cv2
import face_recognition
import numpy as np
import os
from datetime import datetime

In [2]:
path = 'FaceImages'
images = []
classNames = []
myList = os.listdir(path)
print(myList)

['Ciara.jpeg', 'Scarlett Johansson.jpg', 'Ben Affleck.jpeg', 'Pooja Hegde.jpg', 'John Abraham.jpg', 'Paul Rudd.jpg', 'Sunny Deol.jpg', 'Batuhan.jpeg', '.DS_Store', 'Nora Fatehi.jpg', 'Nicole Kidman.jpg', 'Leonardo DiCaprio.jpg', 'Megan Fox.jpg', 'Josh Brolin.jpg', 'Sandra Bullock.jpg', 'Hugh Jackman.jpg', 'Mark Ruffalo.jpg', 'Caitriona.jpeg', 'Tom Cruise.jpg', 'Natalie Portman.jpg', 'Beyonce.jpeg', 'Angelina Jolie.jpg', 'Robert Downey Jr.jpg', 'Denzel Washington.jpg', 'Kareena Kapoor.jpg', 'Kate Winslet.jpg', 'Rihanna.jpeg', 'Priyanka Chopra.jpg', 'Will Smith.jpg', 'Benedict Cumberbatch.jpg', 'Brad Pitt.jpg', 'Jennifer Lawrence.jpg', 'Tom Holland.jpg', '.ipynb_checkpoints', 'Chris Hemsworth.jpg', 'SZA.jpg', 'Brie Larson.jpg', 'Tom Hanks.jpg', 'Chris Evans.jpg', 'Elizabeth Olsen.jpg', 'Gabby.jpeg', 'Vin Diesel.jpg']


In [3]:
for cls in myList:
    img = cv2.imread(f'{path}/{cls}')

    if img is None:
        print(f"Failed to load image: {cls}")
        continue
    
    images.append(img)
    classNames.append(os.path.splitext(cls)[0])
print(classNames)

Failed to load image: .DS_Store
Failed to load image: .ipynb_checkpoints
['Ciara', 'Scarlett Johansson', 'Ben Affleck', 'Pooja Hegde', 'John Abraham', 'Paul Rudd', 'Sunny Deol', 'Batuhan', 'Nora Fatehi', 'Nicole Kidman', 'Leonardo DiCaprio', 'Megan Fox', 'Josh Brolin', 'Sandra Bullock', 'Hugh Jackman', 'Mark Ruffalo', 'Caitriona', 'Tom Cruise', 'Natalie Portman', 'Beyonce', 'Angelina Jolie', 'Robert Downey Jr', 'Denzel Washington', 'Kareena Kapoor', 'Kate Winslet', 'Rihanna', 'Priyanka Chopra', 'Will Smith', 'Benedict Cumberbatch', 'Brad Pitt', 'Jennifer Lawrence', 'Tom Holland', 'Chris Hemsworth', 'SZA', 'Brie Larson', 'Tom Hanks', 'Chris Evans', 'Elizabeth Olsen', 'Gabby', 'Vin Diesel']


In [4]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if img is None:
            print("Empty image detected, skipping...")
            continue
        
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

encodeListKnown = findEncodings(images)
print('Encoding done.')

Encoding done.


In [5]:
def markClockIn(name):
    with open('Clock_In_Sheet.csv', 'r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
            
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name}, {dtString}')

In [6]:
cap = cv2.VideoCapture(0)

In [7]:
import time

timeout_duration = 5  
start_time = time.time()

while True:
    if time.time() - start_time > timeout_duration:
        break
    
    success, img = cap.read()
    if not success:
        break
    
    small_img = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    small_img = cv2.cvtColor(small_img, cv2.COLOR_BGR2RGB)
  
    face_locations = face_recognition.face_locations(small_img)

    face_locations = [(top*4, right*4, bottom*4, left*4) for (top, right, bottom, left) in face_locations]
    
    encodeCurFrame = face_recognition.face_encodings(img, face_locations)
    
    correctly_recognized_faces = 0
    
    for encodeFace, faceLoc in zip(encodeCurFrame, face_locations):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace, tolerance=0.6)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        
        matchIndex = np.argmin(faceDis)
        accuracy = (1 - faceDis[matchIndex]) * 100
        
        if matches[matchIndex] and accuracy >= 50:
            correctly_recognized_faces += 1
            name = classNames[matchIndex].upper()
            y1, x2, y2, x1 = faceLoc
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            
            markClockIn(name)


    total_faces = len(encodeCurFrame)
    accuracy = (correctly_recognized_faces / total_faces) * 100 if total_faces != 0 else 0

    print(f"Total Faces: {total_faces}")
    print(f"Correctly Recognized Faces: {correctly_recognized_faces}")
    print(f"Accuracy: {accuracy:.2f}%")
    
    cv2.imshow('Webcam', img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Total Faces: 1
Correctly Recognized Faces: 0
Accuracy: 0.00%
Total Faces: 1
Correctly Recognized Faces: 0
Accuracy: 0.00%
Total Faces: 1
Correctly Recognized Faces: 0
Accuracy: 0.00%
Total Faces: 1
Correctly Recognized Faces: 1
Accuracy: 100.00%
Total Faces: 1
Correctly Recognized Faces: 0
Accuracy: 0.00%
Total Faces: 1
Correctly Recognized Faces: 0
Accuracy: 0.00%
Total Faces: 1
Correctly Recognized Faces: 0
Accuracy: 0.00%
Total Faces: 1
Correctly Recognized Faces: 1
Accuracy: 100.00%
Total Faces: 1
Correctly Recognized Faces: 1
Accuracy: 100.00%
Total Faces: 1
Correctly Recognized Faces: 1
Accuracy: 100.00%
Total Faces: 1
Correctly Recognized Faces: 0
Accuracy: 0.00%
Total Faces: 1
Correctly Recognized Faces: 1
Accuracy: 100.00%
Total Faces: 1
Correctly Recognized Faces: 1
Accuracy: 100.00%
Total Faces: 1
Correctly Recognized Faces: 1
Accuracy: 100.00%
Total Faces: 1
Correctly Recognized Faces: 1
Accuracy: 100.00%
Total Faces: 1
Correctly Recognized Faces: 1
Accuracy: 100.00%
Total 

Out of the 50 iterations the system performed in 5 seconds, it accurately identified my face 47 times, which I am very happy with.